# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import pickle
import nltk

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix 
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///MessagesCleanDatabase.db')
df = pd.read_sql("SELECT * FROM LabeledMessages", engine)

X = df['message']
y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

print(X.shape)
print(y.shape)

(26216,)
(26216, 36)


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # Normalize text  
    text = text.strip()
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize text 
    tokens = word_tokenize(text) 
    
    # Remove stop words 
    tokens = [t for t in tokens if t not in stopwords.words("english")] 
    
    # Lemmatization - Reduce words to their root form 
    tokens = [WordNetLemmatizer().lemmatize(t) for t in tokens]
    
    # Stemming - Reduce words to their stems
    tokens = [PorterStemmer().stem(t) for t in tokens] 
     
    return tokens

# Testing Output
print(tokenize(X[0]))
print(tokenize(X[1]))
print(tokenize(X[26203]))

['weather', 'updat', 'cold', 'front', 'cuba', 'could', 'pa', 'haiti']
['hurrican']
['bangkok', '24', 'januari', '2012', 'nnt', 'prime', 'minist', 'yingluck', 'shinawatra', 'attend', 'meet', 'perman', 'secretari', 'variou', 'ministri', 'urg', 'quickli', 'distribut', 'flood', 'compens', 'wise', 'util', 'budget']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

# train classifier 
pipeline.fit(X_train, y_train) 

# predict on test data 
y_pred = pipeline.predict(X_test) 

In [6]:
print(y_pred)

[[1 0 0 ..., 0 0 0]
 [1 1 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]
 ..., 
 [1 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]]


In [7]:
print(y_test.shape)
print(y_pred.shape)
print(y.columns.values)

(6554, 36)
(6554, 36)
['related' 'request' 'offer' 'aid_related' 'medical_help'
 'medical_products' 'search_and_rescue' 'security' 'military' 'child_alone'
 'water' 'food' 'shelter' 'clothing' 'money' 'missing_people' 'refugees'
 'death' 'other_aid' 'infrastructure_related' 'transport' 'buildings'
 'electricity' 'tools' 'hospitals' 'shops' 'aid_centers'
 'other_infrastructure' 'weather_related' 'floods' 'storm' 'fire'
 'earthquake' 'cold' 'other_weather' 'direct_report']


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
# pipeline.score(X_test, y_test)

print(classification_report(y_test, y_pred, target_names=y_test.columns))

                        precision    recall  f1-score   support

               related       0.85      0.91      0.88      5057
               request       0.80      0.43      0.56      1138
                 offer       0.00      0.00      0.00        27
           aid_related       0.75      0.60      0.67      2724
          medical_help       0.58      0.07      0.12       543
      medical_products       0.70      0.09      0.17       338
     search_and_rescue       0.57      0.07      0.12       179
              security       0.00      0.00      0.00       112
              military       0.48      0.07      0.11       200
           child_alone       0.00      0.00      0.00         0
                 water       0.91      0.30      0.45       430
                  food       0.83      0.51      0.63       733
               shelter       0.75      0.35      0.47       576
              clothing       0.71      0.19      0.31       103
                 money       0.64      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fdb941ae510>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [10]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estimator__

In [11]:
parameters = {
#     'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__max_df': (0.5, 0.75, 1.0),
#     'vect__max_features': (None, 5000, 10000),
    'tfidf__use_idf': (True, False),
#     'clf__estimator__min_samples_split': [2, 3, 4]
}

cv = GridSearchCV(pipeline, param_grid=parameters)

In [12]:
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__max_df': (0.5, 0.75, 1.0), 'tfidf__use_idf': (True, False)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [13]:
y_impoved_pred = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
print(classification_report(y_test, y_impoved_pred, target_names=y_test.columns))

                        precision    recall  f1-score   support

               related       0.85      0.91      0.88      5057
               request       0.83      0.43      0.57      1138
                 offer       0.00      0.00      0.00        27
           aid_related       0.76      0.62      0.68      2724
          medical_help       0.55      0.07      0.12       543
      medical_products       0.72      0.07      0.12       338
     search_and_rescue       0.56      0.08      0.14       179
              security       0.00      0.00      0.00       112
              military       0.60      0.10      0.18       200
           child_alone       0.00      0.00      0.00         0
                 water       0.86      0.29      0.43       430
                  food       0.84      0.49      0.62       733
               shelter       0.76      0.35      0.48       576
              clothing       0.66      0.18      0.29       103
                 money       0.50      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [15]:
from sklearn.neighbors import KNeighborsClassifier

pipeline_KN = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])

# train classifier 
pipeline_KN.fit(X_train, y_train) 

# predict on test data 
y_pred_KN = pipeline_KN.predict(X_test) 

print(classification_report(y_test, y_pred_KN, target_names=y_test.columns))

                        precision    recall  f1-score   support

               related       0.79      0.98      0.88      5057
               request       0.81      0.11      0.19      1138
                 offer       0.00      0.00      0.00        27
           aid_related       0.78      0.05      0.09      2724
          medical_help       0.00      0.00      0.00       543
      medical_products       0.58      0.02      0.04       338
     search_and_rescue       0.00      0.00      0.00       179
              security       0.00      0.00      0.00       112
              military       0.00      0.00      0.00       200
           child_alone       0.00      0.00      0.00         0
                 water       0.74      0.04      0.08       430
                  food       0.82      0.08      0.14       733
               shelter       0.72      0.05      0.09       576
              clothing       0.83      0.05      0.09       103
                 money       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [16]:
filename = 'finalized_model.sav'
pickle.dump(pipeline, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.